In [1]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_ridge import KernelRidge
import numpy as np
import doubleml as dml
from doubleml.datasets import make_heterogeneous_data

In [169]:
rbf_feature = RBFSampler(gamma=1.0, n_components=100, random_state=123)

.fit_transform(X) transforms each example into random basis representations.

The resulting matrix is $n \times w$, where $n$ is the number of obs and $w$ is the number of random basis

In [171]:
np.random.seed(42)
data_dict = make_heterogeneous_data(
    n_obs=2000,
    p=10,
    support_size=5,
    n_x=1,
    binary_treatment=True,
)
treatment_effect = data_dict['treatment_effect']
data = data_dict['data']
print(type(data))
print(data.columns)
print(data.head())

X = data.filter(like='X').to_numpy()
Y = data.filter(like='y').to_numpy()
print(type(X))
print(X.shape)

<class 'pandas.core.frame.DataFrame'>
Index(['y', 'd', 'X_0', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8',
       'X_9'],
      dtype='object')
          y    d       X_0       X_1       X_2       X_3       X_4       X_5  \
0  4.803300  1.0  0.259828  0.886086  0.895690  0.297287  0.229994  0.411304   
1  5.655547  1.0  0.824350  0.396992  0.156317  0.737951  0.360475  0.671271   
2  1.878402  0.0  0.988421  0.977280  0.793818  0.659423  0.577807  0.866102   
3  6.941440  1.0  0.427486  0.330285  0.564232  0.850575  0.201528  0.934433   
4  1.703049  1.0  0.016200  0.818380  0.040139  0.889913  0.991963  0.294067   

        X_6       X_7       X_8       X_9  
0  0.240532  0.672384  0.826065  0.673092  
1  0.270644  0.081230  0.992582  0.156202  
2  0.289440  0.467681  0.619390  0.411190  
3  0.689088  0.823273  0.556191  0.779517  
4  0.210319  0.765363  0.253026  0.865562  
<class 'numpy.ndarray'>
(2000, 10)


In [179]:
df_t = data[data['d'] == 1]
Y_t  = df_t.filter(like='y').to_numpy()
X_t  = df_t.filter(like='X').to_numpy()
X_t_transform = rbf_feature.fit_transform(X_t)
krr_t = KernelRidge(alpha=1e-5)
krr_t.fit(X_t_transform, Y_t)


df_c = data[data['d'] == 0]
Y_c  = df_c.filter(like='y').to_numpy()
X_c  = df_cfrom scipy.stats.qmc import Sobol

M = 2048
p = 10   # number of covariates
sobol_points = Sobol(d=p, scramble=False).random(M)
.filter(like='X').to_numpy()
X_c_transform = rbf_feature.fit_transform(X_c)
krr_c = KernelRidge(alpha=1e-5)
krr_c.fit(X_c_transform, Y_c)


from scipy.stats.qmc import Sobol
M = 2048
sobol_points = Sobol(d=X.shape[1]).random(M)
# Map Sobol points into RFF feature space
Z_sobol = rbf_feature.transform(sobol_points)

np.mean(np.maximum(0, krr_t.predict(Z_sobol) - krr_c.predict(Z_sobol)))

4.4119220370753744

In [149]:
import numpy as np
from scipy.stats.qmc import Sobol

# Step 1: reconstruct CATE function
def true_tau(x):
    return np.exp(2 * x[:, 0]) + 3 * np.sin(4 * x[:, 1])

# Step 2: Sobol points
p = X.shape[1]
M = 2048
sobol_points = Sobol(d=p).random(M)

# Step 3: evaluate tau
tau_sobol = true_tau(sobol_points)

# Step 4: ReLU + average
W_true = np.mean(np.maximum(0, tau_sobol))
print("True optimal welfare:", W_true)


True optimal welfare: 4.449629628029234


In [2]:
import numpy as np
from scipy.stats.qmc import Sobol
from sklearn.kernel_ridge import KernelRidge
from sklearn.kernel_approximation import RBFSampler
from doubleml.datasets import make_heterogeneous_data

np.random.seed(123)

# ---------------------------
# SIMULATION PARAMETERS
# ---------------------------
rep = 500
M = 1024
p = 2
n = 1500
support_size = 2
gamma = 1.0
alpha = 1e-5
n_components = 100

# ---------------------------
# TRUE CATE FUNCTION
# ---------------------------
def true_tau(x):
    return np.exp(2 * x[:, 0]) + 3 * np.sin(4 * x[:, 1])

# Sobol grid
sobol_points = Sobol(d=p, scramble=False).random(M)
tau_sobol = true_tau(sobol_points)
W_true = np.mean(np.maximum(0, tau_sobol))
print("True optimal welfare =", W_true)

# ---------------------------
# ONE ITERATION
# ---------------------------
def compute_optimal_welfare(n_obs=n, p=p, support_size=support_size,
                            gamma=gamma, alpha=alpha,
                            sobol_points=sobol_points,
                            n_components=n_components):
    
    data_dict = make_heterogeneous_data(
        n_obs=n_obs,
        p=p,
        support_size=support_size,
        n_x=1,
        binary_treatment=True,
    )
    data = data_dict["data"]

    # -------------------- split --------------------
    df_t = data[data["d"] == 1]
    df_c = data[data["d"] == 0]

    Y_t = df_t["y"].to_numpy().ravel()
    X_t = df_t.filter(like="X").to_numpy()

    Y_c = df_c["y"].to_numpy().ravel()
    X_c = df_c.filter(like="X").to_numpy()

    # ----------------- RFF -------------------------
    rff = RBFSampler(gamma=gamma, n_components=n_components, random_state=42)

    B_all = rff.fit_transform(np.vstack([X_t, X_c]))
    B_t = B_all[: len(X_t)]
    B_c = B_all[len(X_t) :]

    # ----------------- linear ridge = approx KRR ---
    krr_t = KernelRidge(alpha=alpha, kernel="linear").fit(B_t, Y_t)
    krr_c = KernelRidge(alpha=alpha, kernel="linear").fit(B_c, Y_c)

    e_t = Y_t - krr_t.predict(B_t)
    e_c = Y_c - krr_c.predict(B_c)     # FIXED

    # ---------------- Sobol evaluation ---------------
    B_int = rff.transform(sobol_points)
    h_int = krr_t.predict(B_int) - krr_c.predict(B_int)

    W_hat = np.mean(np.maximum(0, h_int))

    # -------------- Asymptotic Variance --------------
    ind_good = (h_int >= 0)
    bases    = np.hstack([B_int, -B_int])
    Bun      = bases[ind_good, :].sum(axis=0) / len(bases)
    Bun_t    = Bun[:B_int.shape[1]]
    Bun_c    = Bun[B_int.shape[1]:]
    
    R_t = np.linalg.inv(B_t.T @ B_t + alpha * np.eye(B_t.shape[1])) # R matrix: (B'B + λI)^(-1)
    R_c = np.linalg.inv(B_c.T @ B_c + alpha * np.eye(B_c.shape[1])) 

    Patty_t = R_t @ (B_t.T @  np.diag(e_t**2) @ B_t) @ R_t
    Patty_c = R_c @ (B_c.T @  np.diag(e_c**2) @ B_c) @ R_c

    asy_var = float(Patty_t + Patty_c)
    se      = np.sqrt(asy_var)
    
    return W_hat, se

# ---------------------------
# MONTE CARLO LOOP
# ---------------------------
results = np.zeros(rep)

for r in range(rep):
    results[r] = compute_optimal_welfare()
    if (r+1) % 50 == 0:
        print("Completed:", r+1)

print("\nMC mean =", results.mean())
print("Bias (mean - W_true) =", results.mean() - W_true)
print("Std =", results.std())


True optimal welfare = 4.447681288540517
Completed: 50
Completed: 100
Completed: 150
Completed: 200
Completed: 250
Completed: 300


KeyboardInterrupt: 

In [ ]:
# --- Asymptotic variance ---
    # ind_good = (h0_B_int >= 0)

    # term3 = np.hstack([B_int, -B_int])
    # Bun = term3[ind_good, :].sum(axis=0) / len(term3)
    # Bun_t = Bun[:B_int.shape[1]]
    # Bun_c = Bun[B_int.shape[1]:]

    # R_t = np.linalg.inv(B_t.T @ B_t + alpha * np.eye(B_t.shape[1])) # R matrix: (B'B + λI)^(-1)
    # R_c = np.linalg.inv(B_c.T @ B_c + alpha * np.eye(B_c.shape[1])) # R matrix: (B'B + λI)^(-1)
    

    # Patty_t = R_t @ (B_t.T @  np.diag(e_t**2) @ B_t) @ R_t
    # Patty_c = R_c @ (B_c.T @  np.diag(e_c**2) @ B_c) @ R_c

    #asy_var = float(Bun_t @ Patty_t @ Bun_t + Bun_c @ Patty_c @ Bun_c)
    #se = np.sqrt(asy_var)